In [9]:
# random grid generation

In [10]:
import random
import math

def gridGeneration():
    grid = [[[0,0,0,0] for _ in range(10)] for _ in range(4)]
    betas = list()
    for i in range(10):
        for j in range(12):
            betas.append(i)

    for i in range(0,4):
        alphas = [_ for _ in range(10)]
        for j in range(10):
            random.shuffle(alphas)
            grid[i][j][0] = alphas.pop()
            for k in range(1,4):
                random.shuffle(betas)
                grid[i][j][k] = betas.pop()

    for i in range(0,40):
        for j in range(i+1,40):
            r1 = math.floor(i/10) 
            c1 = i%10
            r2 = math.floor(j/10)
            c2 = j%10
            if grid[r1][c1] == grid[r2][c2]:
                return gridGeneration()
            
    return grid

In [11]:
# some helper functions

In [12]:
def findLocation(row, level, digit):
    loc = -1
    for i,cell in enumerate(row):
        if cell[level] == digit:
            return i
        
    return -1

def moveVector(grid, pin, loc):
    second = grid[loc[0]][loc[1]]
    move_vector = [-1, -1, -1, -1]
    for i in range(4):
        x1 = findLocation(grid[i], 0, pin[i])
        x2 = findLocation(grid[i], 0, second[i])
        if x2 < x2:
            x2 += 10
        move_vector[i] = x2-x1
    return move_vector


def findSet(grid, move_vector):
    P = list()
    for i in range(4):
        for j in range(10):
            derived_pin = [-1, -1, -1, -1]
            for row in range(4):
                for col in range(10):
                    if grid[row][col][0] == grid[i][j][row]:
                        k = (col - move_vector[row] + 10)%10
                        derived_pin[row] = grid[row][k][0]
                        break
            P.append(derived_pin)
            
    return P

def labelEvent(grid, move_vector, actual_pin, ref_cell):
    x = ref_cell[0]
    y = ref_cell[1]
    derived_pin = [-1, -1, -1, -1]
    for i in range(4):
        for j in range(10):
            if grid[i][j][0] == grid[x][y][i]:
                k = (j - move_vector[i] + 10)%10
                derived_pin[i] = grid[i][k][0]
                break
                
    if derived_pin == actual_pin:
        return "CORRECT"
    else:
        return "INCORRECT"

In [13]:
# semi random grid generation

In [14]:
def nextGrid(Si, PIN):
    pins = [Si[0:10], Si[10:20], Si[20:30], Si[30:40]]
    pins = [[(y,i) for y,i in enumerate(x)] for x in pins]
    chosenPins = []
    for x in pins:
        if PIN in x:
            chosenPins.append(PIN)
        else:
            random.shuffle(x)
            chosenPins.append(x.pop())
            
    grid = [[[-1,-1,-1,-1] for _ in range(10)] for _ in range(4)] 
    betas = list()
    for i in range(10):
        for j in range(12):
            betas.append(i)
            
    move_vector = [-1, -1, -1, -1]
    alphas = [_ for _ in range(10)]
    random.shuffle(alphas)
    for i in range(10):
        grid[0][i][0] = alphas.pop()
    x1 = findLocation(grid[0], 0, chosenPins[0][1][0])
    x2 = chosenPins[0][0]
    if x2 < x1:
        x2 += 10
    move_vector[0] = x2-x1
    for j in range(1,4):
        x1 = findLocation(grid[0], 0, chosenPins[j][1][0])
        x2 = (x1+move_vector[0])%10
        ele = grid[0][x2][0]
        grid[j][chosenPins[j][0]][0] = ele
    for i in range(1,4):
        alphas = [_ for _ in range(10)]
        random.shuffle(alphas)
        for cell in grid[i]:
            if cell[0] != -1:
                alphas.remove(cell[0])
        for cell in grid[i]:
            if cell[0] == -1:
                cell[0] = alphas.pop()
    for i in range(1,4):
        nums = [_ for _ in range(10)]
        random.shuffle(nums)
        move_vector[i] = nums.pop()
        for j,p in enumerate(chosenPins):
            x1 = findLocation(grid[i], 0, p[1][i])
            x2 = (x1 + move_vector[i])%10
            ele = grid[i][x2][0]
            grid[j][p[0]][i] = ele
            betas.remove(ele)
            
    for row in range(4):
        for col in range(10):
            for k in range(4):
                if grid[row][col][k] == -1:
                    random.shuffle(betas)
                    grid[row][col][k] = betas.pop()
                     
    return grid

In [17]:
# demo for Si intersection Si+1 >= 4

from tabulate import tabulate

pin = [1,2,3,4]
loc = [1,1] # 0 indexed for the purpose of this algorithm
g1 = gridGeneration()
grid1 = [["".join([str(z) for z in y]) for y in x] for x in g1]
g2 = nextGrid(findSet(g1, moveVector(g1, pin, loc)), (loc[1], pin))
grid2 = [["".join([str(z) for z in y]) for y in x] for x in g2]
s1 = findSet(g1, moveVector(g1, pin, loc))
s2 = findSet(g2, moveVector(g2, pin, loc))

print(tabulate(grid1, tablefmt="grid"))
print()
print()
print(tabulate(grid2, tablefmt="grid"))
print()
print()

for i in range(40):
    if s1[i] == s2[i]:
        print(i, s1[i])

+------+------+------+------+------+------+------+------+------+------+
| 3778 | 8546 | 4444 | 0181 | 6492 | 1114 | 2754 | 7806 | 5174 | 9029 |
+------+------+------+------+------+------+------+------+------+------+
| 1809 | 5746 | 6498 | 8731 | 3283 | 2593 | 7094 | 0672 | 4282 | 9947 |
+------+------+------+------+------+------+------+------+------+------+
| 4553 | 6677 | 3725 | 1052 | 2921 | 7666 | 5015 | 8931 | 0103 | 9293 |
+------+------+------+------+------+------+------+------+------+------+
| 6063 | 5098 | 2837 | 7560 | 3358 | 0321 | 8586 | 4012 | 9395 | 1086 |
+------+------+------+------+------+------+------+------+------+------+


+------+------+------+------+------+------+------+------+------+------+
| 8335 | 5498 | 1135 | 6712 | 9478 | 4976 | 0395 | 2411 | 7462 | 3999 |
+------+------+------+------+------+------+------+------+------+------+
| 9076 | 1362 | 0982 | 4058 | 6815 | 7449 | 2005 | 8923 | 3834 | 5645 |
+------+------+------+------+------+------+------+------+-----